In [19]:
import pandas as pd
import datetime
from tqdm import tnrange

In [20]:
userLog = pd.read_csv("./clean_data/userlog_feature.csv")

In [42]:
userLog.shape

(989830, 12)

In [46]:
userLog['datetime'] = pd.to_datetime(userLog['datetime'])
isTrain = userLog['datetime']<datetime.datetime(2018,2,25)
test = userLog[~isTrain]
userLog = userLog[isTrain]


In [22]:
#testUserAnswerMerged.to_csv("./clean_data/_test_with_train.csv")

In [47]:
userLog1 = userLog.sort_values("datetime")
userLog1.sample()

,userCode,project_id,requestedDevice,userAgent,pageReferrer,year,month,day,hour,datetime,user_clicked_by_hour,user_clicked_projects_by_hour
473763,514802fd-07d7-581c-ddbb-359252f6e916,995,Desktop,Windows,HomeWebsite,2018,1,22,19,2018-01-22 19:00:00,7,1


In [48]:
userLog2VecDf = userLog1.groupby("userCode").agg({"project_id":lambda x: list(x)})

In [49]:
#userLog2VecDf.sample(10)
userLog2VecDf.shape

(409375, 1)

In [50]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [27]:
#userLog2VecDf['project_id'].apply(len).value_counts()#.plot(kind="kde")

In [51]:
moreThanOne = userLog2VecDf['project_id'].apply(len)>1
projs = userLog2VecDf['project_id'][moreThanOne]
projs = [[str(p) for p in proj] for proj in projs]
users = userLog2VecDf.index[moreThanOne]


In [52]:
documents = [TaggedDocument(proj, [user]) for i, (proj,user) in enumerate(zip(projs,users))]

In [53]:
documents[:10]

[TaggedDocument(words=['6709', '6712'], tags=['0000e1e2-f595-0ae7-860f-fcc07dcb116e']),
 TaggedDocument(words=['8829', '4703', '6577', '4703', '8829', '4184'], tags=['0000fa46-1f0b-9504-b568-43479d17620e']),
 TaggedDocument(words=['8587', '5985', '9095'], tags=['00019a11-0aaa-2559-4feb-534724830d3e']),
 TaggedDocument(words=['6070', '6070', '6070', '6070'], tags=['0002abb4-b370-d047-0d70-bdfa59780107']),
 TaggedDocument(words=['8412', '8412', '5452', '5858', '7591', '4411', '3738', '4980', '7128'], tags=['0002d107-28fe-ffb0-2ebf-a7fb1fdb8a3f']),
 TaggedDocument(words=['1828', '1828'], tags=['00031c31-5c7b-563b-4298-f1b320bfea6f']),
 TaggedDocument(words=['3839', '3839'], tags=['00033b74-b212-b7b1-bef1-38b21d2097b7']),
 TaggedDocument(words=['9333', '9333'], tags=['00039df0-f294-5bd1-93d1-7ec49b27940b']),
 TaggedDocument(words=['8631', '8742', '9334', '8742', '9334'], tags=['0003d990-37d2-140c-a68e-060bc1e81ea5']),
 TaggedDocument(words=['6545', '3522', '6545'], tags=['00040647-ef7a-936

### Train

In [54]:
model = Doc2Vec(documents, vector_size=300, window=3, min_count=1, workers=4,epochs=4,sample=1e-4, negative=10)

### Let's see if it works or not

In [32]:
# def recommendByVector(userId):
#     try:
#     #print(most_similar_user)
#         seen = userLog2VecDf.loc[userId].project_id
#         most_similar_user = model.docvecs.most_similar(userId,topn=100)
#         recommending_projs = []
#         for user, similarity in most_similar_user:
#             friend_projects = [i for i in userLog2VecDf.loc[user].project_id if i not in seen]
#             recommending_projs.extend(friend_projects)
#             recommending_projs = list(set(recommending_projs))
#             if len(recommending_projs)>=7: break

#         #     print(userLog2VecDf.loc[user].project_id, similarity)

#         return " ".join([str(i) for i in recommending_projs])
#     except:
#         return "5858 3738 4201 5452 5678 5764 5067"
    
# recommendByVector("0018fd5f-213f-c9ab-ebf4-0dcfb63be43b")
userLog.head()

,userCode,project_id,requestedDevice,userAgent,pageReferrer,year,month,day,hour,datetime,user_clicked_by_hour,user_clicked_projects_by_hour
0,7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2,7956,Mobile,Android,HomeWebsite,2018,1,1,0,2018-01-01,3,3
1,7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2,7956,Mobile,Android,HomeWebsite,2018,1,1,0,2018-01-01,3,3
2,cb5b4b68-cc01-6db6-f54b-4a0f881301c5,5067,Mobile,iPhone,HomeWebsite,2018,1,1,0,2018-01-01,1,1
3,5f74cef2-0d1e-b619-3564-0955a14e0985,6654,Mobile,iPhone,Google,2018,1,1,0,2018-01-01,1,1
4,dba8f279-844e-eef6-73ac-22bd7d1353cc,6474,Mobile,iPad,Google,2018,1,1,0,2018-01-01,2,2


In [33]:
userId = "7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2"
userLog[userLog['userCode']==userId].project_id

0    7956
1    7956
7    7956
Name: project_id, dtype: int64

In [34]:
all_project = model.wv.vocab
def recommendByProjects(userId, userLog=userLog,topn = 50, withProb = False):
    seen = userLog[userLog['userCode']==userId].project_id
    
    seen = [str(i) for i in seen if str(i) in all_project]
    if len(seen) ==0: return "5858 3738 4201 5452 5678 5764 5067"
    most_similar_proj = model.wv.most_similar(positive=seen,topn=100)
    recommending_projs = []
    if withProb: return most_similar_proj
    
    for proj, similarity in most_similar_proj:
        if proj not in seen: recommending_projs.append(proj)
        if len(recommending_projs)>=topn: break

    #     print(userLog.loc[user].project_id, similarity)
    
    
    return " ".join([str(i) for i in recommending_projs])

recommendByProjects("7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2", withProb =False)

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'4140 3054 5721 3546 8281 4728 5603 3706 6517 5827 5058 5820 8465 3462 5291 4776 3418 8078 3943 3805 6539 5391 5032 6544 1782 6265 4998 4859 6575 8726 2565 5563 6797 4366 3912 3821 7337 3479 4917 3256 2611 4002 4783 3960 2878 3252 5177 5866 4103 3972'

## Test and evaluation

In [55]:
testing = pd.read_csv("./data/testing_users.csv")

In [56]:
ans = testing['userCode'].apply(recommendByProjects)
testing['project_id'] = ans
testing.to_csv("./submission/item2vec.csv",index=False)

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [35]:
testUserAnswer = test.groupby("userCode").agg({"project_id":lambda x: list(x)})
testUserAnswer.columns = ["ans_project_list"]
testUserAnswerMerged = userLog2VecDf.merge(testUserAnswer,on='userCode',right_index=True)

In [36]:
# testing = testUserAnswerMerged.reset_index()
ans = testing['userCode'].apply(recommendByProjects)
testing['preds'] = ans
testing['ans_project'] = testing['ans_project_list'].apply(lambda x: list(set(x)))
testing['MAP'] = testing.apply(lambda x: score_AP(x['ans_project'],x['preds']),axis=1)
print("MAP@7",testing['MAP'].mean())#0.0299286 by 4 epochs 300dim, and 10 negative

NameError: name 'testing' is not defined

In [41]:
scores = []
for i in tnrange(5):
    testing_users = pd.read_csv("./clean_data/test_data/test_before_021718_700row_fold{}.csv".format(i))
    testing_users.head()

    testing_users['project_id'] = testing_users['userCode'].apply(recommendByProjects)
    testing_users['ans_project_id'] = testing_users['ans_project_id'].apply(eval)
    #testing_users['ans_project'] = testing_users['ans_project_id'].apply(lambda x: list(set(x)))
    testing_users['MAP'] = testing_users.apply(lambda x: score_AP(x['ans_project_id'],x['project_id']),axis=1)
    score = testing_users['MAP'].mean()
    print(score)
    scores.append(score)
print(sum(scores)/len(scores))  #0.04469187

HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Exception in thread Thread-44:
Traceback (most recent call last):
  File "/home/users/u5722792744_1/miniconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/home/users/u5722792744_1/miniconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



0.04063417232659163



KeyboardInterrupt: 

In [ ]:
testing_users.head()

In [ ]:
print("Accuracy: ",testing['MAP'].apply(lambda x: x>0).sum()/testing.shape[0])

In [39]:
def score_AP(correct, recommended, n=7):
    recommended = recommended.split(" ")
    assert(len(set(recommended))==len(recommended)) # Recommended should be unique    
    precisions = []
    correct_score = 1
    for idx, recommend_item in enumerate(recommended):
        try:
            if int(recommend_item) not in correct: precisions.append(0)
            else:                         
                precisions.append(correct_score/(idx+1))
                correct_score+=1
        except:
            print(recommend_item)
    return sum(precisions)/min(len(correct),n)
#score_AP(["5858", "5678"],"5678 5858 8058 8107 6527 9132 6446",7)

In [ ]:
testing.head().apply(lambda x: score_AP(x['ans_project_list'],x['preds']),axis=1)

In [ ]:
submission_user = pd.read_csv("./data/testing_users.csv")
userLogForSubmit = pd.read_csv("./clean_data/userlog_feature.csv")
userLogNewest = userLogForSubmit.groupby("userCode").agg({"project_id":lambda x: list(x)})

In [ ]:
submission_user['project_id'] = submission_user['userCode'].apply(lambda x: recommendByProjects(x,userLogNewest))

In [ ]:
submission_user.head()

In [ ]:
submission_user.to_csv("./submission/item2vec_v4.csv",index=False)

In [ ]:
model.save("./model/doc2vec_before_train.wv")

In [37]:
model.wv.syn0

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.


array([[-5.0508118e-01, -3.3308697e-01, -3.0090153e-01, ...,
        -1.5204206e-01,  2.9880059e-01, -3.2322603e-01],
       [-2.9613972e-01, -3.2852197e-01, -6.7148632e-01, ...,
         3.7514459e-02,  2.3710223e-01, -1.0929456e-01],
       [-6.4490771e-01, -2.1913674e-01, -3.6538839e-01, ...,
        -1.7875176e-03,  1.5475969e-01, -1.1890832e-01],
       ...,
       [ 2.4966253e-03, -6.8143458e-04, -2.9621189e-03, ...,
         1.0150285e-03, -5.7258839e-03,  3.4133758e-04],
       [ 1.4497744e-02, -6.2584970e-03, -4.1974881e-03, ...,
         1.9449983e-02, -2.2434704e-02, -1.4009111e-02],
       [ 9.8351762e-03, -3.3049991e-03, -3.0553872e-03, ...,
         1.2795477e-02, -1.6126435e-02, -5.8805025e-03]], dtype=float32)